In [15]:
# code in this block is taken from https://github.com/y-z-zhang/SBD/blob/master/sbd.py
import numpy as np
from numpy import linalg as la
from scipy.linalg import block_diag
from scipy.stats import ortho_group

############### PURPOSE #################
# SBD is a simple and efficient algorithm for finding a (often finest) simultaneous block diagonalization of an arbitrary number of symmetric matrices.
# The algorithm works by finding the eigendecomposition of a single matrix, which is a random linear combination of all the matrices to be simultaneously block diagonalized.
# The current algorithm is inspired by the results presented in K. Murota et al. Jpn. J. Ind. Appl. Math 27, 125–160 (2010).
# For an efficient algorithm that also guarantees the optimal SBD, see https://github.com/y-z-zhang/net-sync-sym.
# There, you can also find concrete examples on how the SBD algorithm can be applied to analyze cluster synchronization patterns in complex networks.

############### USEAGE #################
# [P,BlockSizes] = sbd(A,threshold)
# A --- list containing the set of matrices to be simultaneously block diagonalized
# threshold --- real number, matrix entries below threshold are considered zero
# P --- orthogonal transformation matrix that performs SBD on A
# BlockSizes --- array listing the size of each common block

############### REFERENCE #################
# Y. Zhang, V. Latora, and A. E. Motter,
# Unified treatment of synchronization patterns in generalized networks with higher-order, multilayer, and temporal interactions,
# Commun. Phys. 4, 195 (2021).

def sbd(A,threshold):
	n = len(A[0])	# size of the matrices to be simultaneously block diagonalized
	m = len(A)		# number of matrices to be simultaneously block diagonalized
	BlockSizes = []	# initialize the array that lists the size of each common block

    # B is a random self-adjoint matrix generated by matrices from A (and their conjugate transposes)
	B = np.zeros((n,n))
	for p in range(m):
		B = B + np.random.normal()*(A[p]+A[p].transpose())

    # find the eigenvalues and eigenvectors of B
	D, V = la.eigh(B)

    # C is a matrix used to sort the column vectors of V (i.e., the base vectors)
    # such that the base vectors corresponding to the same common block are next to each other
	C = np.zeros((n,n))
	for p in range(m):
		C = C + np.random.normal()*(A[p]+A[p].transpose())
	C = V.transpose()@C@V
	#for p in range(m):
	#	C = C + np.random.normal()*V.transpose()@A[p]@V

    # arrays used to track which base vectors have been sorted
	remaining_basis = list(range(n))
	sorted_basis = []

    # the sorting process: find C_ij's that are nonzero and group the base vectors v_i and v_j together
	while len(remaining_basis) > 0:
		current_block = [remaining_basis[0]]
		current_block_size = 1
		if len(remaining_basis) > 1:
			for idx in remaining_basis[1:]:
				if np.abs(C[remaining_basis[0],idx]) > threshold:
					current_block.append(idx)
					current_block_size = current_block_size + 1

		for idx in current_block:
			sorted_basis.append(idx)
			remaining_basis.remove(idx)

		# do the following in case there are zero entries inside the block
		current_block_extra = []
		if len(remaining_basis) > 0:
			for idx in remaining_basis:
				for ind in current_block:
					if np.abs(C[ind,idx]) > threshold:
						current_block_extra.append(idx)
						current_block_size = current_block_size + 1
						break

		for idx in current_block_extra:
			sorted_basis.append(idx)
			remaining_basis.remove(idx)

		BlockSizes.append(current_block_size)

    # the sorted base vectors give the final orthogonal/unitary transformation matrix that performs SBD on A
	P = V[:,sorted_basis]

	return P, BlockSizes

## uncomment below for an example use of the algorithm
n = 10	# size of the matrices to be simultaneously block diagonalized
## A1 and A2 are random matrices that share a predefined block structure
A1 = block_diag(np.random.rand(3,3),np.random.rand(2,2),np.random.rand(4,4),np.random.rand(1,1))
A2 = block_diag(np.random.rand(3,3),np.random.rand(2,2),np.random.rand(4,4),np.random.rand(1,1))
## transform A1 and A2 using a random orthogonal matrix Q
Q = ortho_group.rvs(dim=n)
A1 = Q.T@A1@Q
A2 = Q.T@A2@Q
A = [A1,A2]
## use the SBD algorithm to uncover the common block structure
sbd(A,1e-5)


(array([[ 0.6683208 ,  0.6051395 , -0.0117798 ,  0.03058645, -0.00647833,
          0.09848303,  0.11968723,  0.06775071,  0.39659947, -0.011489  ],
        [-0.02265862, -0.10893264, -0.02099649,  0.29851985,  0.07976337,
         -0.40774013, -0.05821065,  0.47713036,  0.19983069, -0.67414905],
        [ 0.15312225,  0.05924615,  0.36748022, -0.30849642,  0.46634167,
          0.35487587, -0.17493674, -0.1207555 , -0.33505594, -0.49190756],
        [-0.2047555 ,  0.17737872, -0.35999714, -0.6132519 ,  0.42577559,
         -0.17478982, -0.0412953 ,  0.39763123,  0.11144926,  0.19199733],
        [ 0.17589606, -0.16893677, -0.44205011,  0.01197911,  0.04276721,
          0.18892856,  0.74328657,  0.08024607, -0.3420727 , -0.17753316],
        [ 0.12640569,  0.14935411,  0.17714159,  0.25532951, -0.06498149,
          0.118156  , -0.16383053,  0.65473502, -0.53892922,  0.31779887],
        [-0.06661896,  0.37295364, -0.51464229, -0.10594199, -0.4417407 ,
         -0.00641723, -0.3887944

In [16]:
# define the so(8) basis
from toqito.states import basis

n = 8
# get the column vectors
dim_8_ket = [basis(n, 0), basis(n, 1), basis(n, 2), basis(n, 3), basis(n, 4), basis(n, 5), basis(n, 6), basis(n, 7)]
# get the bra
dim_8_bra = []
for i in range(n):
    item = dim_8_ket[i]
    dim_8_bra.append(item.conj().T)

so8_basis = []
for i in range(n):
    for k in range(1, n):
        if i < k:
            m_ik = -1j*(np.outer(dim_8_ket[i], dim_8_ket[k])-np.outer(dim_8_ket[k], dim_8_ket[i]))
            so8_basis.append(m_ik)

# print basis
print("so(8) basis \n")
j = 1
for i in so8_basis:
    print("Basis Element", j, "\n", i, "\n")
    j= j+1


so(8) basis 

Basis Element 1 
 [[0.-0.j 0.-1.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.+1.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]] 

Basis Element 2 
 [[0.-0.j 0.-0.j 0.-1.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.+1.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]] 



In [17]:
print("Block structure of so(8)")
for i, mat in enumerate(so8_basis):
    bd_struct = sbd(mat,1e-5)[-1]
    print("Basis Element", i, " ",bd_struct)

Block structure of so(8)
Basis Element 0   [3, 1, 1, 1, 1, 1]
Basis Element 1   [4, 1, 1, 1, 1]
Basis Element 2   [4, 1, 1, 1, 1]
Basis Element 3   [4, 1, 1, 1, 1]
Basis Element 4   [4, 1, 1, 1, 1]
Basis Element 5   [4, 1, 1, 1, 1]
Basis Element 6   [8]
Basis Element 7   [3, 1, 1, 1, 1, 1]
Basis Element 8   [4, 1, 1, 1, 1]
Basis Element 9   [4, 1, 1, 1, 1]
Basis Element 10   [4, 1, 1, 1, 1]
Basis Element 11   [4, 1, 1, 1, 1]
Basis Element 12   [7, 1]
Basis Element 13   [3, 1, 1, 1, 1, 1]
Basis Element 14   [4, 1, 1, 1, 1]
Basis Element 15   [4, 1, 1, 1, 1]
Basis Element 16   [4, 1, 1, 1, 1]
Basis Element 17   [6, 1, 1]
Basis Element 18   [3, 1, 1, 1, 1, 1]
Basis Element 19   [4, 1, 1, 1, 1]
Basis Element 20   [4, 1, 1, 1, 1]
Basis Element 21   [5, 1, 1, 1]
Basis Element 22   [3, 1, 1, 1, 1, 1]
Basis Element 23   [4, 1, 1, 1, 1]
Basis Element 24   [4, 1, 1, 1, 1]
Basis Element 25   [3, 1, 1, 1, 1, 1]
Basis Element 26   [3, 1, 1, 1, 1, 1]
Basis Element 27   [2, 1, 1, 1, 1, 1, 1]


In [18]:

# define the so(7) basis
from toqito.states import basis

n = 7
# get the column vectors
dim_7_ket = [basis(n, 0), basis(n, 1), basis(n, 2), basis(n, 3), basis(n, 4), basis(n, 5), basis(n, 6)]
# get the bra
dim_7_bra = []
for i in range(n):
    item = dim_7_ket[i]
    dim_7_bra.append(item.conj().T)

so7_basis = []
for i in range(n):
    for k in range(1, n):
        if i < k:
            m_ik = -1j*(np.outer(dim_7_ket[i], dim_7_ket[k])-np.outer(dim_7_ket[k], dim_7_ket[i]))
            so7_basis.append(m_ik)

# print basis
print("so(7) basis \n")
j = 1
for i in so7_basis:
    print("Basis Element", j, "\n", i, "\n")
    j= j+1

so(7) basis 

Basis Element 1 
 [[0.-0.j 0.-1.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.+1.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]] 

Basis Element 2 
 [[0.-0.j 0.-0.j 0.-1.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.+1.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]] 

Basis Element 3 
 [[0.-0.j 0.-0.j 0.-0.j 0.-1.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j]
 [0.+1.j 0.-0.j 0.-0.j 0.-0.j 0.-0.j 0.-0.

In [19]:
print("Block structure of so(7)")
for i, mat in enumerate(so7_basis):
    bd_struct = sbd(mat,1e-5)[-1]
    print("Basis Element", i, " ",bd_struct)

Block structure of so(7)
Basis Element 0   [3, 1, 1, 1, 1]
Basis Element 1   [4, 1, 1, 1]
Basis Element 2   [4, 1, 1, 1]
Basis Element 3   [4, 1, 1, 1]
Basis Element 4   [4, 1, 1, 1]
Basis Element 5   [7]
Basis Element 6   [3, 1, 1, 1, 1]
Basis Element 7   [4, 1, 1, 1]
Basis Element 8   [4, 1, 1, 1]
Basis Element 9   [4, 1, 1, 1]
Basis Element 10   [6, 1]
Basis Element 11   [3, 1, 1, 1, 1]
Basis Element 12   [4, 1, 1, 1]
Basis Element 13   [4, 1, 1, 1]
Basis Element 14   [5, 1, 1]
Basis Element 15   [4, 1, 1, 1]
Basis Element 16   [4, 1, 1, 1]
Basis Element 17   [4, 1, 1, 1]
Basis Element 18   [3, 1, 1, 1, 1]
Basis Element 19   [3, 1, 1, 1, 1]
Basis Element 20   [2, 1, 1, 1, 1, 1]
